1. Zassanie zdjęć do testów. Dodanie do nich czarnej ramki. Znalezienie tylko pikselów istotnych(nieczarnych). Wycięcie masek 30 x 30 z centralnym pikselem nieczarnym.

2. Podpięcie sieci ConvNet do przetestowania wycietych masek ( sieć trenowana przez 0-1TrainSetGenerator dzięki dostarczonym pociętym maskom 30 x 30 z oryginalnych zdjęć)


In [2]:
import skimage as ski
from skimage import data, io, filters, exposure
from skimage.filters import rank
from skimage import img_as_float, img_as_ubyte
from skimage.morphology import disk
import skimage.morphology as mp
from skimage import util
from skimage.color import rgb2hsv, hsv2rgb, rgb2gray
from skimage.filters.edges import convolve
from PIL import Image, ImageOps
from PIL import Image
import random
import glob
        
### Util of photo_with_border_maker 
def add_border(input_image):
    border=(15,15,15,15)
    img = Image.open(input_image)
    if isinstance(border, int) or isinstance(border, tuple):
        bimg = ImageOps.expand(img, border=border)
    else:
        raise RuntimeError('Border is not an integer or tuple!')
    bimg.save(input_image)

### Add border to original photo and make sibling for drawing white vessels
def sibling_maker(name):
    add_border(name)
    tmp = ski.io.imread(name)
    img = Image.new('RGB', (tmp.shape[1],tmp.shape[0]), (0,0,0))
    img.save("4Final/toGo/"+str(name[12:-4])+".png", "PNG")

### Crop the area from the original where non black pixel detected
def crop(image_path, coords, saved_location):
    #coords -> A tuple of x/y coordinates (x1, y1, x2, y2)
    #photo = img_as_float(ski.io.imread(name)) # old method
    image_obj = Image.open(image_path)
    cropped_image = image_obj.crop(coords)
    cropped_image.save(saved_location)
    #cropped_image.show()

### Look for non-black pixels
def check(name):
    idd = name[-7:-4]
    img = rgb2gray(img_as_float(io.imread(name)))
    print("IMG SHAPE:",img.shape)
    for xx,row in enumerate(img):
        for yy,pixel in enumerate(row):
            if(yy%2 == 0):
                continue
            else:
                if(pixel != 0):
                    if xx < 100:
                        xxx = '0'+str(xx)
                    else:
                        xxx=xx
                    if yy < 100:
                        yyy = '0'+str(yy)
                    else:
                        yyy=yy
                    #Maska 30x30 jeżeli nie mamy czarnego elementu w xx i yy    
                    crop(name, (xx-15, yy-15, xx+15, yy+15), '32DevidedInputTest/' + str(idd)+'_x_'+str(xxx)+'_y_'+str(yyy)+'.jpg') 


ImportError: DLL load failed: The specified module could not be found.

In [1]:
file_names = [fn for fn in glob.glob("31InputTest/*")]
print("Znaleziono {} zdjęć do przerobienia".format(len(file_names)))
for nr,name in enumerate(file_names):
    print(nr+1,name)
    #sibling_maker - Make result photo for drawing 'vessels hit' (black in the begining) with the same shape as @name photo
    #sibling_maker(name)
    check(name)

NameError: name 'glob' is not defined

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import glob

# dimensions of our images.
img_width, img_height = 30,30

train_data_dir = '2Trening/train'
validation_data_dir = '2Trening/validation'
nb_train_samples = 12000
nb_validation_samples = 2400
epochs = 150
batch_size = 100

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('sigmoid'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(48, (3, 3), input_shape=input_shape))
model.add(Activation('sigmoid'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator()

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save('my_model2.h5')